# Demo No. 1: Hello binary!


This set of tutorials will guide you trough the basic structure of this package. This particular demo will show you how to build your binary system and access its basic parameters.

## Import correct modules

Objects modelled in our first binary system are divided into two groups:
* system - binary system,
* compoents - stars.

Each of those object types have to be imported separatelly using the following imports:   

In [1]:
from elisa.binary_system.system import BinarySystem
from elisa.base.star import Star

## Setting up the components

To avoid any mass confusion, we will use astropy.units module to specify units for each parameter where possible.  

In [2]:
from astropy import units as u
from elisa import units

`units` module in elisa package stores deafult units that ELISa uses internally. Eg.:

In [3]:
units.DISTANCE_UNIT, units.PERIOD_UNIT

(Unit("m"), Unit("d"))

Now we can initialize components with their respective parameters. Lets define the primary and secondary component:

In [4]:
primary = Star(
    mass=2.15 * u.solMass,
    surface_potential=3.6,
    synchronicity=1.0,
    t_eff=10000 * u.K,
    gravity_darkening=1.0,
    discretization_factor=5,  # angular size (in degrees) of the surface elements
    albedo=0.6,
    metallicity=0.0,
)

secondary = Star(
    mass=0.45 * u.solMass,
    surface_potential=5.39,
    synchronicity=1.0,
    t_eff=8000 * u.K,
    gravity_darkening=1.0,
    albedo=0.6,
    metallicity=0,
)

Component parameters can be also set and modified separatelly using setters:

In [5]:
primary.synchronicity = 1.0

and accesed via getters:

In [6]:
primary.synchronicity

1.0

As soon as the components are defined, we can set up BinarySystem instance:

In [14]:
bs = BinarySystem(
    primary=primary,
    secondary=secondary,
    argument_of_periastron=58 * u.deg,
    gamma=-30.7 * u.km / u.s,
    period=2.5 * u.d,
    eccentricity=0.0,
    inclination=85 * u.deg,
    primary_minimum_time=2440000.00000 * u.d,
    phase_shift=0.0,
)


2020-05-28 14:15:19,300 - 27647 - binary_system.system - INFO: initialising object BinarySystem
2020-05-28 14:15:19,373 - 27647 - binary_system.system - INFO: setting discretization factor of secondary component to 20.00 according to discretization factor of the primary component.


where again, parameters can be accesed and overwriten using getters and setters

In [15]:
bs.eccentricity = 0.1
bs.eccentricity

0.1

Just to make sure that the system with new parameters is physically valid and to recalculate all the dependent variables such as mass ratio, semi-major axis, etc., we have to reinitialize the binary system instance `bs`:

In [16]:
bs.init()

2020-05-28 14:15:28,903 - 27647 - binary_system.system - INFO: initialising object BinarySystem
2020-05-28 14:15:28,962 - 27647 - binary_system.system - INFO: setting discretization factor of secondary component to 20.00 according to discretization factor of the primary component.


## Accesing parameters

Finally, after binary system initialization, we can acces basic properties of our system such as semi-major axis,  morphology, and radii:

In [17]:
bs.semi_major_axis*units.DISTANCE_UNIT.to(u.solRad)  # conversion to solar radii

10.659024832175282

morphology of the system can be also displayed:

In [18]:
bs.morphology

'detached'

along with radii of the components in default SMA (semi major axis) units:

In [19]:
print(f'Polar radius: {primary.polar_radius}')
print(f'Side radius: {primary.side_radius}')
print(f'Backward radius: {primary.backward_radius}')
print(f'Forward radius: {primary.forward_radius}')

Polar radius: 0.29391569126942896
Side radius: 0.29861794003120684
Backward radius: 0.3015396415348499
Forward radius: 0.3034650095475252


Values for critical potentials for each component can be accessed using these getters:

In [20]:
print(f'Critical potential for primary component: {primary.critical_surface_potential}')
print(f'Critical potential for secondary component: {secondary.critical_surface_potential}')

Critical potential for primary component: 2.4269724166520685
Critical potential for secondary component: 2.440275836628986


*Note that critical potential values are not the same due to eccentricity is set on value 0.1. Those values are critical potentials during periastron.